<a href="https://colab.research.google.com/github/ApurbaPaul-NLP/Machine-Learning/blob/main/03_09_2022_Building_a_Basic_Sentiment_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd

In [2]:
path='/content/drive/MyDrive/TENSORLFOW/combined_data.csv'

dataset=pd.read_csv(path)
sentences=dataset['text'].tolist()
labels=dataset['sentiment'].tolist()

In [3]:
training_size=int(len(sentences)*0.8)
training_size

1593

In [4]:
training_sentences=sentences[0:training_size]
testing_sentences=sentences[training_size:]
training_labels=labels[0:training_size]
testing_labels=labels[training_size:]

In [5]:
training_labels_final=np.array(training_labels)
testing_labels_final=np.array(testing_labels)

In [6]:
vocab_size=1000
embedding_dim=16
max_length=100
# truc_type='post'
# padding_type='post'
oov_tok='<OOV>'

In [7]:
tokenizer=Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index=tokenizer.word_index
sequences=tokenizer.texts_to_sequences(training_sentences)
padded=pad_sequences(sequences,maxlen=max_length,padding='post',truncating='post')
testing_sequences=tokenizer.texts_to_sequences(testing_sentences)
testing_padded=pad_sequences(testing_sequences,maxlen=max_length,padding='post',truncating='post')

In [14]:
reverse_word_index=dict([(value,key) for key,value in word_index.items()])
def decode_review(text):
  return ' '.join([reverse_word_index.get(i,'*') for i in text])

print(decode_review(padded[1])) 
print(training_sentences[1]) 

good case excellent value * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
Good case Excellent value.


In [15]:
model=tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(6,activation='relu'),
      tf.keras.layers.Dense(1,activation='sigmoid')
])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 16)           16000     
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dense (Dense)               (None, 6)                 9606      
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 25,613
Trainable params: 25,613
Non-trainable params: 0
_________________________________________________________________


In [16]:
num_epochs=10
model.fit(padded,training_labels_final,epochs=num_epochs,validation_data=(testing_padded,testing_labels_final))

Epoch 1/10
50/50 [==============================] - 4s 8ms/step - loss: 0.6924 - accuracy: 0.5179 - val_loss: 0.6929 - val_accuracy: 0.5213
Epoch 2/10
50/50 [==============================] - 0s 5ms/step - loss: 0.6821 - accuracy: 0.5675 - val_loss: 0.6999 - val_accuracy: 0.4110
Epoch 3/10
50/50 [==============================] - 0s 4ms/step - loss: 0.6494 - accuracy: 0.5938 - val_loss: 0.6801 - val_accuracy: 0.4837
Epoch 4/10
50/50 [==============================] - 0s 5ms/step - loss: 0.5534 - accuracy: 0.8023 - val_loss: 0.5821 - val_accuracy: 0.7143
Epoch 5/10
50/50 [==============================] - 0s 5ms/step - loss: 0.4084 - accuracy: 0.8964 - val_loss: 0.5616 - val_accuracy: 0.6842
Epoch 6/10
50/50 [==============================] - 0s 4ms/step - loss: 0.2993 - accuracy: 0.9190 - val_loss: 0.4840 - val_accuracy: 0.7694
Epoch 7/10
50/50 [==============================] - 0s 4ms/step - loss: 0.2318 - accuracy: 0.9410 - val_loss: 0.4505 - val_accuracy: 0.7845
Epoch 8/10
50/50 [==

In [18]:
e=model.layers[0]
weights=e.get_weights()[0]
print(weights.shape)

(1000, 16)


In [19]:
import io

# Write out the embedding vectors and metadata
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [20]:
# Download the files
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
# Use the model to predict a review   
fake_reviews = ['I love this phone', 'I hate spaghetti', 
                'Everything was cold',
                'Everything was hot exactly as I wanted', 
                'Everything was green', 
                'the host seated us immediately',
                'they gave us free chocolate cake', 
                'not sure about the wilted flowers on the table',
                'only works when I stand on tippy toes', 
                'does not work when I stand on my head']

print(fake_reviews) 

# Create the sequences
padding_type='post'
sample_sequences = tokenizer.texts_to_sequences(fake_reviews)
fakes_padded = pad_sequences(sample_sequences, padding=padding_type, maxlen=max_length)           

print('\nHOT OFF THE PRESS! HERE ARE SOME NEWLY MINTED, ABSOLUTELY GENUINE REVIEWS!\n')              

classes = model.predict(fakes_padded)

# The closer the class is to 1, the more positive the review is deemed to be
for x in range(len(fake_reviews)):
  print(fake_reviews[x])
  print(classes[x])
  print('\n')

# Try adding reviews of your own
# Add some negative words (such as "not") to the good reviews and see what happens
# For example:
# they gave us free chocolate cake and did not charge us

['I love this phone', 'I hate spaghetti', 'Everything was cold', 'Everything was hot exactly as I wanted', 'Everything was green', 'the host seated us immediately', 'they gave us free chocolate cake', 'not sure about the wilted flowers on the table', 'only works when I stand on tippy toes', 'does not work when I stand on my head']

HOT OFF THE PRESS! HERE ARE SOME NEWLY MINTED, ABSOLUTELY GENUINE REVIEWS!

I love this phone
[0.99151796]


I hate spaghetti
[0.05468401]


Everything was cold
[0.37260595]


Everything was hot exactly as I wanted
[0.79855335]


Everything was green
[0.55911875]


the host seated us immediately
[0.8155644]


they gave us free chocolate cake
[0.7938059]


not sure about the wilted flowers on the table
[0.03003904]


only works when I stand on tippy toes
[0.9763455]


does not work when I stand on my head
[0.00849257]


